In [20]:
import pandas as pd
from libvoikko import Voikko
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.nn import softmax
from keras.models import Sequential 
from keras.layers import Dense, BatchNormalization, Embedding, LSTM, Flatten, Dropout, Conv1D, GlobalMaxPooling1D, Bidirectional
from keras.metrics import Accuracy
from keras import utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np

#### Read data from csv

In [2]:
data = pd.read_csv("iotech2.csv",index_col=0)

In [3]:
data

,Titles,Category
0,Kysymyksiä & Vastauksia,0
1,Omat autot -esittelythread! (KESKUSTELUA VAIN ...,0
2,Omat autot -keskusteluketju! Yleinen ja offtop...,0
3,Mikä ärsyttää kanssa-autoilijoissa?,0
4,Taasko se romu on hajalla? Täällä korjataan au...,0
...,...,...
385,Valvontakamera ulkokäyttöön,4
386,Kotivakuutuksen enimmäiskorvauksien arviointi,4
387,Paras alle 200e maksava ulkovalvontakamera?,4
388,Höyrymopit ja lattiapesurit,4


#### Select inputs and labels

In [4]:
texts = data['Titles']
labels = data['Category']

#### Set max number of words in corpus and the max length of sentence (title).

In [5]:
Voikko.setLibrarySearchPath('D:\Libvoikko')
v=Voikko(language=u'fi',path='D:\Libvoikko')

In [6]:
def base_form(text):
    line=''
    txt = text.lower().replace(".", "").replace(",", "")
    word_list = txt.split(" ")
    for w in word_list:
        voikko_dict = v.analyze(w)
    #Extract the base form, if the word is recognized
        if voikko_dict:
            bf_word = voikko_dict[0]['BASEFORM']
    #If word is not recognized, add the original word
        else:
            bf_word = w

        line+=bf_word+' '
    return line

In [7]:
data['Titles'] = data['Titles'].apply(lambda word:base_form(word))

In [8]:
data

,Titles,Category
0,kysymys & vastaus,0
1,oma Autto -esittelythread! (keskustelua vain e...,0
2,oma Autto -keskusteluketju! yleinen ja offtopi...,0
3,mikä ärsyttää kanssa-autoilijoissa?,0
4,taas se romu olla hajalla? täällä korjata auto,0
...,...,...
385,valvontakamera ulkokäyttö,4
386,kotivakuutus enimmäiskorvaus arviointi,4
387,hyvä alla 200e maksava ulkovalvontakamera?,4
388,höyrymoppi ja lattiapesuri,4


In [9]:
NUM_WORDS = 10000
MAX_TEXT_LEN = 12
BATCH_SIZE=64

##### Create corpus

In [10]:
tokenizer =keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS)
tokenizer.fit_on_texts(texts)

In [11]:
tokenizer

In [12]:
word_count = 10
keys = list(tokenizer.word_index.keys())[:word_count]
values = list(tokenizer.word_index.values())[:word_count]
list(zip(keys, values))

[('ja', 1),
 ('f1', 2),
 ('gp', 3),
 ('2016', 4),
 ('2017', 5),
 ('auton', 6),
 ('the', 7),
 ('keskustelua', 8),
 ('onko', 9),
 ('yms', 10)]

In [13]:
sequences = tokenizer.texts_to_sequences(texts)
X = keras.utils.pad_sequences(sequences, maxlen=MAX_TEXT_LEN)  # Standardization of the number of elements in a vector
y = labels.copy()



In [14]:
print(X)

[[  0   0   0 ...   0 535 985]
 [  0   0   0 ... 466 987 536]
 [  0   0 465 ... 989 110 492]
 ...
 [  0   0   0 ... 233 234 235]
 [  0   0   0 ... 236   1 237]
 [  0   0   0 ...   0  17 238]]


#### Create training, validation and testing datasets

In [15]:
X, X_test, y, y_test = train_test_split(X, y, test_size=0.05,random_state=107)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=107)
print(f'Training sample size: {len(X_train)}')
print(f'Test sample size: {len(X_test)}')
print(f'Val sample size: {len(X_val)}')

Training sample size: 2127
Test sample size: 160
Val sample size: 912


In [16]:
X_train.shape

(2127, 12)

In [17]:
len(y.unique())

5

#### Neuarl networks for classification

In [21]:
# LSTM model for classification.
model = Sequential()
model.add(Embedding(NUM_WORDS, 32, input_length=MAX_TEXT_LEN))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(10, return_sequences=True)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(LSTM(8))
model.add(Dense(5))

# CNN just for reference. Not suited for text classification
model2 = Sequential()
model2.add(Embedding(NUM_WORDS, 64, input_length=MAX_TEXT_LEN))
model2.add(Dropout(0.2))
model2.add(Conv1D(32,
                 5,
                 padding='same',
                 activation='relu',
                 strides=1))
# we use max pooling:
model2.add(GlobalMaxPooling1D())
model2.add(Dropout(0.2))
model2.add(Dense(32,activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(5))



In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 12, 32)            320000    
                                                                 
 dropout_1 (Dropout)         (None, 12, 32)            0         
                                                                 
 bidirectional (Bidirectiona  (None, 12, 20)           3440      
 l)                                                              
                                                                 
 batch_normalization (BatchN  (None, 12, 20)           80        
 ormalization)                                                   
                                                                 
 dropout_2 (Dropout)         (None, 12, 20)            0         
                                                                 
 lstm_1 (LSTM)               (None, 8)                

In [23]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 12, 64)            640000    
                                                                 
 dropout_3 (Dropout)         (None, 12, 64)            0         
                                                                 
 conv1d (Conv1D)             (None, 12, 32)            10272     
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dropout_4 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                      

#### Creating callback to stop training proses when val_loss stops decreasing
#### Also creating class weights to balance data inbalances

In [24]:
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=5, verbose=1,restore_best_weights=True),  
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, min_delta=0.01, min_lr=1e-10, patience=3, verbose=1, mode='auto')
]
class_weights={0:1.,1:1.5,2:1.5,3:1.,4:1.5}

In [25]:
model.compile(metrics=['accuracy'], loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='rmsprop')

#### Training the model

In [26]:
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=40, class_weight=class_weights, callbacks=callbacks,validation_data=(X_val,y_val))

Epoch 1/40
34/34 [==============================] - 5s 34ms/step - loss: 1.7821 - accuracy: 0.4673 - val_loss: 1.5785 - val_accuracy: 0.4989 - lr: 0.0010
Epoch 2/40
34/34 [==============================] - 0s 9ms/step - loss: 1.3149 - accuracy: 0.7015 - val_loss: 1.5379 - val_accuracy: 0.3399 - lr: 0.0010
Epoch 3/40
34/34 [==============================] - 0s 9ms/step - loss: 0.9823 - accuracy: 0.7870 - val_loss: 1.4930 - val_accuracy: 0.4309 - lr: 0.0010
Epoch 4/40
34/34 [==============================] - 0s 9ms/step - loss: 0.7287 - accuracy: 0.8505 - val_loss: 1.4271 - val_accuracy: 0.5406 - lr: 0.0010
Epoch 5/40
34/34 [==============================] - 0s 9ms/step - loss: 0.5503 - accuracy: 0.8994 - val_loss: 1.3641 - val_accuracy: 0.5570 - lr: 0.0010
Epoch 6/40
34/34 [==============================] - 0s 9ms/step - loss: 0.4230 - accuracy: 0.9304 - val_loss: 1.2800 - val_accuracy: 0.6634 - lr: 0.0010
Epoch 7/40
34/34 [==============================] - 0s 9ms/step - loss: 0.3127 - 

#### Crating some function to tests the trained model

In [27]:
class_names=[
"Prosessorit, ylikellotus, emolevyt ja muistit",
"Näytönohjaimet",
"Tallennus",
"Kotelot ja virtalähteet",
"Jäähdytys",
"Konepaketit",
"Kannettavat tietokoneet",
"Buildit, setupit, kotelomodifikaatiot & DIY",
"Oheislaitteet ja muut PC-komponentit"
]
class_names2=[
"Autot ja liikenne",
"Urheilu",
"TV- & nettisarjat, elokuvat ja musiikki",
"Ruoka & juoma",
"Koti ja asuminen",
]

In [28]:
class_names2

['Autot ja liikenne',
 'Urheilu',
 'TV- & nettisarjat, elokuvat ja musiikki',
 'Ruoka & juoma',
 'Koti ja asuminen']

In [29]:
predictions=model.predict(X_test)
score = softmax(predictions[0])
print(
    "This text most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names2[np.argmax(score)], 100 * np.max(score))
)


5/5 [==============================] - 1s 2ms/step
This text most likely belongs to Autot ja liikenne with a 41.79 percent confidence.


In [30]:
print(predictions[0])

[ 0.97064465 -0.61894864 -1.1904014  -0.0404917   0.6277128 ]


In [31]:
def ml_pipeline(text: str,ml_model) -> str:
    """LSTM model prediction function for this sample"""
    try:
        text=base_form(text)
        print(text)
        sequence = tokenizer.texts_to_sequences([text])
        sequence = keras.utils.pad_sequences(sequence, maxlen=MAX_TEXT_LEN)
        if sequence.max() == 0:
            return 'Enter the words in English'
        else:
            predict = ml_model.predict(sequence, verbose=0)
            score = softmax(predict)
            print(predict)
            return class_names2[np.argmax(score)]
        
    except AttributeError as AE:
        print(AE)
        return 'Enter the text'



In [36]:
 ml_pipeline('Lentokoneella vai junalla',model)

lentokone vai juna 
[[ 1.0279696  -0.2977997  -0.41759133 -0.9751146  -0.21358784]]


'Autot ja liikenne'

In [33]:
data['Category'].value_counts()

3    908
0    903
2    581
1    417
4    390
Name: Category, dtype: int64